<a href="https://colab.research.google.com/github/federgargiulo/Vibration-AI/blob/main/Asciugacapelli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importo le librerie 

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical, normalize, plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
import scikeras
from scikeras.wrappers import KerasClassifier

In [ ]:

!pip install scikeras
!pip install q keras==2.10.0

In [2]:
#Importo il dataset e definisco il numero di input e di classi di output

AC = pd.read_excel('/content/AscVel.xlsx')


In [3]:
#Splitto il dataset (Opzionale)
#X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train,test_size=0.2)


In [4]:
#Preparo il dataset per addestrare la rete

X_train = AC.loc[:,'Ax':'fz']
Y_train = AC.loc[:,'Labels']
Y_train = to_categorical(Y_train)

In [ ]:
#Addestro il modello

model = Sequential()
model.add(Dense(25,input_dim = 6,activation = 'relu'))
model.add(Dense(25,activation = 'relu'))
model.add(Dense(3,activation = 'softmax'))
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.fit(X_train,Y_train,epochs = 100,batch_size = 5,verbose = 1)


In [ ]:
#prediction = model.predict(X_test)
#print(prediction[:].round())
#print(Y_test)

#Valuto l'accuracy del modello e ne stampo il plot
model.evaluate(X_train,Y_train)
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#Salvo il modello e lo converto(salvo sia in .h5 che in tflite)

model.save('modello_asc' + '.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimization = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
open('modello_asc' + '.tflite','wb').write(tflite_model)

In [6]:
#Definisco il modello da usare per la grid_search

def define_model(neurons = 10,learning_rate = 0.001, momentum = 0.3):
    my_model = Sequential()
    my_model.add(Dense(neurons,activation = 'relu',kernel_initializer = 'uniform',input_dim = 6))
    my_model.add(Dense(neurons,activation = 'relu',kernel_initializer = 'uniform'))
    my_model.add(Dense(3,activation = 'softmax',kernel_initializer = 'uniform'))
    optimizer = SGD(lr = learning_rate, momentum = momentum)
    my_model.compile(loss = 'categorical_crossentropy',optimizer = optimizer,metrics = ['accuracy'])
    return my_model     

In [ ]:
#Tuning degli iperparametri

batch_size = 16
epochs = 100
my_model = KerasClassifier(neurons = 1,learning_rate = 0,momentum = 0,build_fn = define_model,epochs = epochs, batch_size = batch_size, verbose = 1)
learning_rate = [0.001,0.01,0.1]
momentum = [0.3,0.7,0.9]
neurons = [10,11,12,13,14,15]
param_grid = dict(learning_rate=learning_rate,momentum=momentum,neurons = neurons)
grid = GridSearchCV(estimator=my_model, param_grid = param_grid, n_jobs=16, cv =3)
grid.fit(X_train, Y_train) 
